In [1]:
using DrWatson
@quickactivate "Doran_etal_2022"
using SPI
using NewickTreeTools

In [2]:
# tiplabels = string.([:A, :B, :C, :D, :E])
dij = calc_spi_mtx(rand(100,50))
hc = hclust(dij, linkage=:average, branchorder=:optimal);

In [3]:
SPI.nwstr(hc);
NewickTree.nwstr(newick(hc));
@time SPI.nwstr(hc);
@time NewickTree.nwstr(newick(hc));

  0.000079 seconds (1.10 k allocations: 161.438 KiB)
  0.000448 seconds (4.88 k allocations: 479.797 KiB)


In [4]:
# tiplabels = string.([:A, :B, :C, :D, :E])
dij = calc_spi_mtx(rand(20000,50))
hc = hclust(dij, linkage=:average, branchorder=:optimal);

In [6]:
@time SPI.nwstr(hc);
@time NewickTree.nwstr(newick(hc));

  0.026278 seconds (220.00 k allocations: 47.406 MiB, 17.02% gc time)
  0.184846 seconds (983.84 k allocations: 137.337 MiB, 23.76% gc time)
